In [4]:
#coding:utf-8
import math
import sys
from collections import defaultdict

class NaiveBayes:
    """Multinomial Naive Bayes"""
    def __init__(self):
        self.categories = set()     # カテゴリの集合
        self.vocabularies = set()   # ボキャブラリの集合
        self.wordcount = {}         # wordcount[cat][word] カテゴリでの単語の出現回数
        self.catcount = {}          # catcount[cat] カテゴリの出現回数
        self.denominator = {}       # denominator[cat] P(word|cat)の分母の値
    
    def train(self, data):
        """ナイーブベイズ分類器の訓練"""
        # 文書集合からカテゴリを抽出して辞書を初期化
        for d in data:
            cat = d[0]
            self.categories.add(cat)
        for cat in self.categories:
            self.wordcount[cat] = defaultdict(int)
            self.catcount[cat] = 0
        # 文書集合からカテゴリと単語をカウント
        for d in data:
            cat, doc = d[0], d[1:]
            self.catcount[cat] += 1
            for word in doc:
                self.vocabularies.add(word)
                self.wordcount[cat][word] += 1
        # 単語の条件付き確率の分母の値をあらかじめ一括計算しておく（高速化のため）
        for cat in self.categories:
            self.denominator[cat] = sum(self.wordcount[cat].values()) + len(self.vocabularies)
    
    def classify(self, doc):
        """事後確率の対数 log(P(cat|doc)) がもっとも大きなカテゴリを返す"""
        best = None
        max = -sys.maxsize
        for cat in self.catcount.keys():
            p = self.score(doc, cat)
            if p > max:
                max = p
                best = cat
        return best
    
    def wordProb(self, word, cat):
        """単語の条件付き確率 P(word|cat) を求める"""
        # ラプラススムージングを適用
        # wordcount[cat]はdefaultdict(int)なのでカテゴリに存在しなかった単語はデフォルトの0を返す
        # 分母はtrain()の最後で一括計算済み
        return float(self.wordcount[cat][word] + 1) / float(self.denominator[cat])
    
    def score(self, doc, cat):
        """文書が与えられたときのカテゴリの事後確率の対数 log(P(cat|doc)) を求める"""
        total = sum(self.catcount.values())  # 総文書数
        score = math.log(float(self.catcount[cat]) / total)  # log P(cat)
        for word in doc:
            # logをとるとかけ算は足し算になる
            score += math.log(self.wordProb(word, cat))  # log P(word|cat)
        return score
    
    def __str__(self):
        total = sum(self.catcount.values())  # 総文書数
        return "documents: %d, vocabularies: %d, categories: %d" % (total, len(self.vocabularies), len(self.categories))


# Introduction to Information Retrieval 13.2の例題
data = [["yes", "Chinese", "Beijing", "Chinese"],
        ["yes", "Chinese", "Chinese", "Shanghai"],
        ["yes", "Chinese", "Macao"],
        ["no", "Tokyo", "Japan", "Chinese"]]
# ナイーブベイズ分類器を訓練
nb = NaiveBayes()
nb.train(data)
print (nb)
print ("P(Chinese|yes) = ", nb.wordProb("Chinese", "yes"))
print ("P(Tokyo|yes) = ", nb.wordProb("Tokyo", "yes"))
print ("P(Japan|yes) = ", nb.wordProb("Japan", "yes"))
print ("P(Chinese|no) = ", nb.wordProb("Chinese", "no"))
print ("P(Tokyo|no) = ", nb.wordProb("Tokyo", "no"))
print ("P(Japan|no) = ", nb.wordProb("Japan", "no"))
# テストデータのカテゴリを予測
test = ["Chinese", "Chinese", "Chinese", "Tokyo", "Japan"]
print ("log P(yes|test) =", nb.score(test, "yes"))
print ("log P(no|test) =", nb.score(test, "no"))
print (nb.classify(test))


documents: 4, vocabularies: 6, categories: 2
P(Chinese|yes) =  0.42857142857142855
P(Tokyo|yes) =  0.07142857142857142
P(Japan|yes) =  0.07142857142857142
P(Chinese|no) =  0.2222222222222222
P(Tokyo|no) =  0.2222222222222222
P(Japan|no) =  0.2222222222222222
log P(yes|test) = -8.10769031284391
log P(no|test) = -8.906681345001262
yes


In [9]:
import numpy as np
from sklearn.naive_bayes import GaussianNB # ガウシアン
X = np.array([[1,2,3,4,5,6,7,8],
              [1,1,3,4,5,6,6,7],
              [2,1,2,4,5,8,8,8]]) # 特徴ベクトル
y = np.array([1, 2, 3]) # そのラベル
t = np.array([2,2,4,5,6,8,8,8]) # テストデータ

clf = GaussianNB() # 正規分布を仮定したベイズ分類
clf.fit(X, y) # 学習をする
print(clf.predict(t))

from sklearn.naive_bayes import BernoulliNB #ベルヌーイ
clf = BernoulliNB()
clf.fit(X, y)
print(clf.predict(t) )


from sklearn.naive_bayes import MultinomialNB #多項分布
clf = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
clf.fit(X, y)
print(clf.predict(t))

[3]
[1]
[2]


/Users/arimuraterutoshi/.pyenv/versions/anaconda3-2.4.1/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/arimuraterutoshi/.pyenv/versions/anaconda3-2.4.1/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/arimuraterutoshi/.pyenv/versions/anaconda3-2.4.1/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using 